<a href="https://colab.research.google.com/github/n-aghaebrahim/Probability-stats-for-AI/blob/final/Wine_Data_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import math
import scipy.stats
from scipy.stats import norm
import statistics
import seaborn as sns
from sklearn.linear_model import LinearRegression
import requests
import io
%matplotlib inline
import pandas_profiling as pp

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import ttest_ind
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from google.colab import drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
#2. Get the file
#make sure you upload all your data files to your Google drive and change share->Advanced->change->anyone with the link can view
downloaded = drive.CreateFile({'id':'13cmkIvE7Kql6sP7Jy0DEIHDZ8AbAlxE6'}) # replace the id with id of file you want to access
downloaded.GetContentFile('winequality-red.csv')

downloaded = drive.CreateFile({'id':'15I3Vo5KmJOZ7p21RB0zLaGzMrlWYsM3P'}) # replace the id with id of file you want to access
downloaded.GetContentFile('winequality-white.csv')


## Pre-processing Functions

In [4]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def preprocess_df(df, target_col, test_size=0.2, random_state=0):
    """
    Preprocesses a pandas dataframe by standard scaling the features and splitting
    the data into training and test sets using cross validation.

    Parameters:
        df (pd.DataFrame): The input dataframe to be preprocessed.
        target_col (str): The name of the target column in the dataframe.
        test_size (float, optional): The size of the test set as a fraction of the data. Default is 0.2.
        random_state (int, optional): The seed for the random number generator used in train_test_split. Default is 0.

    Returns:
        train_df (pd.DataFrame): The preprocessed training set features and target column.
        test_df (pd.DataFrame): The preprocessed test set features and target column.
    """
    # Separate the target column from the features
    X = df.drop(columns=[target_col])
    y = df[target_col]

    # Standardize the features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    return X_train, X_test, y_train, y_test
    


## Add Wine Categorization to column

In [5]:
red_wine = pd.read_csv('winequality-red.csv',sep=';')
white_wine = pd.read_csv('winequality-white.csv', sep=';')

In [6]:
#Create Wine category column for red = 0 & white = 1
red_wine['Red0_White1'] = 0

# shift column 'Name' to first position
last_column = red_wine.pop('quality')
  
# insert column using insert(position,column_name,
# first_column) function
red_wine.insert(12, 'quality', last_column)
red_wine.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Red0_White1,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,0,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0,5


In [7]:
#Create Wine category column for red = 0 & white = 1
white_wine['Red0_White1'] = 1

# shift column 'Name' to first position
last_column = white_wine.pop('quality')
  
# insert column using insert(position,column_name,
# first_column) function
white_wine.insert(12, 'quality', last_column)
white_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Red0_White1,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,1,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,1,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,1,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,1,6


### Combine Dataframes together

In [8]:
print(red_wine.shape, white_wine.shape)

(1599, 13) (4898, 13)


In [9]:
# Combine Dataframes together
frames = [red_wine, white_wine]

wine_df = pd.concat(frames).reset_index()
wine_df = wine_df.drop(columns='index')
#wine_df = wine_df

print(wine_df.shape)
wine_df.sample(5)

(6497, 13)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Red0_White1,quality
5946,7.1,0.23,0.30,2.6,0.034,62.0,148.0,0.99121,3.03,0.56,11.3,1,7
2557,6.6,0.20,0.32,1.1,0.039,25.0,78.0,0.99260,3.39,0.54,10.2,1,7
5521,6.7,0.31,0.30,2.4,0.038,30.0,83.0,0.98867,3.09,0.36,12.8,1,7
4215,6.5,0.18,0.41,14.2,0.039,47.0,129.0,0.99678,3.28,0.72,10.3,1,7
1356,6.1,0.34,0.25,1.8,0.084,4.0,28.0,0.99464,3.36,0.44,10.1,0,5


In [10]:
wine_df.shape

(6497, 13)

### Train/Test Split

In [11]:
X_train, X_test, y_train, y_test = preprocess_df(wine_df, 'quality', test_size=0.2, random_state=0)

In [12]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(5197, 12) (1300, 12) (5197,) (1300,)
